In [1]:
import re
import os
import pandas as pd
import numpy as np
import nltk
import wget
import random

from nltk.corpus import stopwords
from ufal.udpipe import Model, Pipeline
from collections import Counter

Установим утилиту для лемматизации, зададим конфигурацию корпусу стоп-слов в русском языке и установим обработку модели на GPU

In [2]:
udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
modelfile = 'udpipe_syntagrus.model'
if not os.path.isfile(modelfile):
        wget.download(udpipe_url)

nltk.download("stopwords")
sw = stopwords.words("russian")
with open('textdata/swm.txt', 'r', encoding='UTF-8') as swm_file:
    swm = swm_file.read().split()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Переформатируем текст так, чтобы в нём остались только предложения на русском

In [3]:
SENT_SEPS = ['\.', '!', '\?', '\n']

def apply_filters(original_text):
    filtered_text = re.sub("[а-яА-ЯёЁ]*[a-zA-Z][^\[]*\[", '', original_text)
    filtered_text = re.sub("\.\]", '', original_text)
    filtered_text = re.sub("франц\.", '', filtered_text)
    filtered_text = re.sub("нем\.", '', filtered_text)
    filtered_text = re.sub("Ред\.\]", '', filtered_text)
    filtered_text = re.sub("-", ' ', filtered_text)
    filtered_text = re.sub("[^а-яА-ЯёЁ  \-\n.!?]", '', filtered_text)
    filtered_text = re.sub("[  ].[  ]", ' ', filtered_text)
    
    return filtered_text
    
def get_sents(text):
    filtered_text = ' '.join(text.split())
    pattern = '|'.join(SENT_SEPS)
    split_text = re.split(pattern, text)
    sentences = [s.strip() for s in split_text if s]
    
    sentences = [re.sub(u'\xa0', ' ', elem) for elem in '\n'.join(sentences).splitlines() if len(elem)>0]
    return sentences

def get_words(text):
    filtered_text = re.sub("[^а-яА-ЯёЁ\-  \n]", '', text)
    filtered_text = re.sub("\n", ' ', filtered_text)
    filtered_text = re.sub("[ ]+", ' ', filtered_text)
    filtered_text = ' '.join(filtered_text.split())
    
    return filtered_text

In [4]:
with open('textdata/ViM.txt', 'r', encoding='ANSI') as original_file:
    original_text = original_file.read()

filtered_text = apply_filters(original_text)

sents = get_sents(filtered_text)

words = get_words('\n'.join(sents))

Зададим функции обработки предложений, из которых мы будем убирать следующие слова:


1. Все части речи, которые несут в себе мало смысловой нагрузки (их теги определены в BANNED_TAGS).


2. Все слова, которые содержатся в корпусе стоп-слов.


3. Все слова длиной не более одного символа (чтобы убрать конструкции типа "и т д", "т е").

In [5]:
# BANNED_TAGS = ['_NUM', '_DET', '_CCONJ', '_SCONJ', '_PART', '_PRON']

def rem_tag(word):
    return re.sub("_[A-Z]*", '', word)

def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken

def clean_token(token, misc):
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token

def clean_lemma(lemma, pos):
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma

def process(pipeline, text='Строка', keep_pos=False, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn


def tag_ud(text='Строка', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        wget.download(udpipe_model_url)

    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    filtered = []
    for line in text:
        output = process(process_pipeline, text=line)
        
        # output = [elem for elem in output if not any(banned in elem for banned in BANNED_TAGS)]
        
        for i, elem in enumerate(output):
            word = rem_tag(elem)
            if (word in sw) or (len(word) <= 1):
                del output[i]
        
        
        filtered.append(' '.join(output))
        
    return filtered

In [6]:
processed = tag_ud(text=sents, modelfile=modelfile)
tokens = ' '.join(processed).split()

Оставим только те слова, что встречаются более 5 раз

In [7]:
word_counts = Counter(tokens)
words = [word for word in tokens if word_counts[word] > 5]

Сохраним обработанный текст:

In [8]:
with open('textdata/words.txt', 'w', encoding='utf-8') as words_file:
    words_file.write('\n'.join(words))

Посмотрим на итоговый размер текста и словаря:

In [9]:
print("Total # of words: {}".format(len(words)))
print("# of unique words: {}".format(len(set(words)))) 

Total # of words: 270168
# of unique words: 5185


На этом предобработка текста завершена и можно переходить к самой модели Word2Vec.